# Let's use our pipeline to preprocess our raw data, first without embedding/vectorization to check out the results

In [1]:
%load_ext autoreload 
%autoreload 2

# imports and setup
import pandas as pd
import os 

# import and instanciate config file to get access to paths
os.chdir(os.path.dirname(os.getcwd()))
from config.config import Config
config = Config()

# import utils functions
from src.pipelines.preprocessor import Preprocessor
from src.pipelines.preproc.pipeline_02 import build_preprocessor_pipeline_02
from sklearn import set_config; set_config(display='diagram')

In [2]:
df_init = pd.read_csv(config.DATA_PROCESSED_PATH + 'books_processed.csv')
df_init

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Plato,"﻿The Project Gutenberg EBook of Sophist, by Pl..."
2,Twain Mark,﻿The Project Gutenberg EBook of On the Decay o...
3,Shakespeare William,"﻿The Project Gutenberg eBook of Pericles, by W..."
4,Shakespeare William,﻿The Project Gutenberg eBook of As You Like It...
...,...,...
431,Twain Mark,﻿The Project Gutenberg EBook of Life On The Mi...
432,Twain Mark,﻿The Project Gutenberg EBook of A Tramp Abroad...
433,Austen Jane,﻿The Project Gutenberg eBook of Sense and Sens...
434,Shakespeare William,﻿\nProject Gutenberg Etext of The Rape of Lucr...


In [3]:
# verify that we have only our 10 authors
df_init['Author'].value_counts()

Author
Shakespeare William                    160
Twain Mark                             115
Dickens Charles                         47
Tolstoy Leo graf                        28
Plato                                   27
Wilde Oscar                             19
Montgomery L. M. (Lucy Maud)            12
Homer                                   12
Austen Jane                             12
Fitzgerald F. Scott (Francis Scott)      4
Name: count, dtype: int64

In [4]:
preprocessor = Preprocessor()
pipeline = build_preprocessor_pipeline_02()
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('text_pipeline',
                                                  Pipeline(steps=[('body',
                                                                   FunctionTransformer(func=<bound method Preprocessor.body_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x1074a6080>>)),
                                                                  ('clean_body',
                                                                   FunctionTransformer(func=<bound method Preprocessor.clean_body_preprocessor of <src.pipelines.pre...
                                                                   FunctionTransformer(func=<bound method Preprocessor.encode_label_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x1074a6080>>))]),
                                                  ['Author'])])),
                ('functiontransformer-1',
                 FunctionTransformer(func=<bound method Preprocessor.chunk_text_preprocessor_nd of <src.pipelines.preprocessor.Preprocessor object at 0x1074a6080>>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<bound method Preprocessor.padding_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x1074a6080>>))])

In [5]:
# let's run our pipeline on our dataframe
df = pipeline.fit_transform(df_init)

In [12]:
df

,Author,Book
0,8,"produced by betsie bush, chuck greif, martin p..."
1,8,which he sees his own life reflected and set d...
2,8,"believed it would, and asked me if i meant to ..."
3,8,"tradition, this one or another--geoffrey cleme..."
4,8,of clemenses who said they had examined the re...
...,...,...
92892,5,"one another, and give rise to a similar opposi..."
92893,5,"she desires to create, but only in what will p..."
92894,5,"good and their opposites, which is true and co..."
92895,5,"medicine which art prescribes for them, and of..."


# We performed a disaggregation from a 436 rows representing each a book, to 92k rows representing each a book's chunk

In [6]:
# save df to csv 
df.to_csv(os.path.join(config.DATA_FINAL_PATH + 'books_processed_pipe02.csv'), index=False)

# Now let's try adding word2vec embedding to our pipeline and do the same